<a href="https://colab.research.google.com/github/harsha-9977/AIML/blob/main/image_caption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch pillow fastapi uvicorn python-multipart


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image, UnidentifiedImageError
import requests
import torch
from io import BytesIO

# Load model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").eval()

def generate_caption(image_path_or_url):
    try:
        if image_path_or_url.startswith("http"):
            headers = {"User-Agent": "Mozilla/5.0"}
            response = requests.get(image_path_or_url, headers=headers)
            response.raise_for_status()  # Ensure request is okay

            # Confirm content-type is image
            content_type = response.headers.get("content-type")
            if "image" not in content_type:
                raise ValueError("URL does not point to an image")

            image = Image.open(BytesIO(response.content)).convert("RGB")

        else:
            image = Image.open(image_path_or_url).convert("RGB")

        inputs = processor(images=image, return_tensors="pt")
        with torch.no_grad():
            output = model.generate(**inputs)

        return processor.decode(output[0], skip_special_tokens=True)

    except UnidentifiedImageError:
        return "❌ Could not identify image format"
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [8]:
url = "https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg"
caption = generate_caption(url)
print("📝 Caption:", caption)


📝 Caption: two pup sitting in a field of flowers


In [2]:
from transformers import BlipProcessor, Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO
import torch

# Load BLIP-2 model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

def generate_blip2_caption(image_url, prompt=""):
    headers = {"User-Agent": "Mozilla/5.0"}
    image = Image.open(BytesIO(requests.get(image_url, headers=headers).content)).convert("RGB")

    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu", torch.float16)
    output = model.generate(**inputs, max_new_tokens=50)
    return processor.decode(output[0], skip_special_tokens=True)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
url = "https://images.hdqwalls.com/download/loki-god-of-mischief-4k-ur-1280x1024.jpg"

print("🖼️ No Prompt:", generate_blip2_caption(url))
print("🖼️ With Prompt:", generate_blip2_caption(url, prompt="Describe this in poetic language."))


🖼️ No Prompt: loki in avengers infinity war

🖼️ With Prompt: Describe this in poetic language.



In [4]:

url = "https://4kwallpapers.com/images/wallpapers/the-wild-robot-2560x1440-19415.jpg"

print("🖼️ No Prompt:", generate_blip2_caption(url))
print("🖼️ With Prompt:", generate_blip2_caption(url, prompt="Describe this in poetic language."))


🖼️ No Prompt: a robot standing in the woods with trees and animals

🖼️ With Prompt: Describe this in poetic language.

